# Init

In [1]:
import numpy as np
import qcodes
import time
import sweep

from qcodes.instrument_drivers.tektronix.Keithley_2400 import Keithley_2400
from qcodes import VisaInstrument

In [2]:
sweep.set_basedir(r"D:\Dropbox (DGG Lab)\VTI\Data\aaron\QH03-11")

## Thermometry

In [3]:
class Lakeshore_335(VisaInstrument):
    # TODO: PID.
    def __init__(self, name, address, **kwargs):
        super().__init__(name, address, terminator='\r\n', **kwargs)
        self.add_parameter('sample_temperature', get_cmd='KRDG? A', get_parser=float, unit='K')
        self.add_parameter('vaporizer_temperature', get_cmd='KRDG? B', get_parser=float, unit='K')
        self.add_parameter('sample_setpoint', get_cmd='SETP? 1', get_parser=float, set_cmd='SETP 1,{:.4f}', unit='K')
        self.add_parameter('vaporizer_setpoint', get_cmd='SETP? 2', get_parser=float, set_cmd='SETP 2,{:.4f}', unit='K')
        self.add_parameter('sample_range', get_cmd='RANGE? 1', get_parser=int, set_cmd='RANGE 1,{}')
        self.add_parameter('vaporizer_range', get_cmd='RANGE? 2', get_parser=int, set_cmd='RANGE 2,{}')
        self.connect_message()
        
    def off(self):
        self.sample_range(0)
        self.vaporizer_range(0)
    
    def smart_set(self, temperature):
        self.sample_setpoint(temperature)
        self.vaporizer_setpoint(temperature * 0.95)
        r = 3
        if temperature < 10:
            r = 1
        elif temperature < 100:
            r = 2
        self.sample_range(r)
        self.vaporizer_range(r)
        
heater = Lakeshore_335('l335_1', 'GPIB0::10::INSTR')
heater.sample_temperature()

Connected to: LSCI 335 (serial:LSA2FM7/#######, firmware:1.4) in 0.15s


1.697

## Keithleys

In [4]:
k_si = Keithley_2400('k_si', 'GPIB0::2::INSTR')
k_si.mode.set('VOLT')
k_si.volt.set(0)
k_si.rangev.set(21)
k_si.rangei.set(1e-6)
k_si.compliancev.set(21)
k_si.compliancei.set(1e-6)
k_si.output.set(1)

k_si.volt.step = 0.002

k_tg = Keithley_2400('k_tg', 'GPIB0::3::INSTR')
k_tg.mode.set('VOLT')
k_tg.volt.set(0)
k_tg.rangev.set(21)
k_tg.rangei.set(1e-6)
k_tg.compliancev.set(15)
k_tg.compliancei.set(1e-6)
k_tg.output.set(1)

k_tg.volt.step = 0.002

k_gg = Keithley_2400('k_gg', 'GPIB0::4::INSTR')
k_gg.mode.set('VOLT')
k_gg.volt.set(0)
k_gg.rangev.set(21)
k_gg.rangei.set(1e-6)
k_gg.compliancev.set(12)
k_gg.compliancei.set(1e-6)
k_gg.output.set(1)

k_gg.volt.step = 0.002

Connected to: KEITHLEY INSTRUMENTS INC. 2400 (serial:4077183, firmware:C32   Oct  4 2010 14:20:11/A02  /U/K) in 0.05s
Connected to: KEITHLEY INSTRUMENTS INC. 2400 (serial:4076931, firmware:C32   Oct  4 2010 14:20:11/A02  /U/K) in 0.04s
Connected to: KEITHLEY INSTRUMENTS INC. 2400 (serial:1299232, firmware:C30   Mar 17 2006 09:29:29/A02  /K/J) in 0.04s


In [10]:
k_gg.volt(0)

## FBL

In [ ]:
channels = [1, 2, 15]
gains = [1e4]*2 + [1e8]
print(channels, gains)
print(len(channels), len(gains))

# Example station

In [ ]:
s = sweep.Station()
s.fp(heater.sample_temperature)
s.fp(heater.vaporizer_temperature)
s.fp(k_tg.curr)
s.fp(k_si.curr)
s.follow_fbl(channels, gains=gains)

s.plot(k_tg.volt, k_tg.curr)
s.plot(k_tg.volt, k_si.curr)
for c in channels:
    s.plot(k_tg.volt, f'fbl_c{c}_x')

#s.sweep(k_tg.volt, np.linspace(0, 0.1, 11), delay=1)
#k_tg.volt(0)